In [1]:
import os
# import streamlit as st
import requests
# from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup
# from sqlalchemy import create_engine, text
from datetime import datetime
import pandas as pd  
# import aiohttp
# import asyncio
import time

import glob
from jinja2 import Environment, FileSystemLoader

# import pdfkit


In [2]:
# Функция для парсинга таблиц участников и волонтёров для конкретного забега
def parse_participant_and_volunteer_tables(run_protocol_link):
    response = requests.get(run_protocol_link)
    soup = BeautifulSoup(response.text, 'html.parser')
    all_tables = soup.find_all('table')

    # Первая таблица – участники
    participants_data = []
    participant_table = all_tables[0]
    for row in participant_table.find_all('tr')[1:]:
        cells = row.find_all('td')
        if len(cells) >= 4:
            position = cells[0].get_text(strip=True)
            name_tag = cells[1].find('a')
            name = name_tag.get_text(strip=True) if name_tag else '—'
            profile_link = name_tag['href'] if name_tag else '—'
            participant_id = profile_link.split('/')[-1] if profile_link != '—' else '—'
            # Количество финишей и волонтёрств
            stats_div = cells[1].find('div', class_='user-stat')
            finishes = '—'
            volunteers = '—'
            if stats_div:
                stats_spans = stats_div.find_all('span')
                finishes = stats_spans[0].get_text(strip=True).split(' ')[0] if len(stats_spans) > 0 else '—'
                volunteers = stats_spans[1].get_text(strip=True).split(' ')[0] if len(stats_spans) > 1 else '—'
            # Клубы
            club_tags = cells[1].find_all('span', class_='club-icon')
            clubs = ', '.join([club['title'] for club in club_tags]) if club_tags else '—'
            # Возрастная группа и Age Grade
            age_group = cells[2].get_text(strip=True).split(' ')[0] if cells[2] else '—'
            age_grade_tag = cells[2].find('div', class_='age_grade')
            age_grade = age_grade_tag.get_text(strip=True) if age_grade_tag else '—'
            # Время и достижения
            time = cells[3].get_text(strip=True) if cells[3] else '—'
            achievements = []
            achievements_div = cells[3].find('div', class_='table-achievments')
            if achievements_div:
                achievement_icons = achievements_div.find_all('span', class_='results_icon')
                for icon in achievement_icons:
                    achievements.append(icon['title'])  # Описание достижения
            participants_data.append([position, name, profile_link, participant_id, clubs, finishes, volunteers, age_group, age_grade, time, ', '.join(achievements)])
            
    # Вторая таблица – волонтёры
    volunteers_data = []
    volunteer_table = all_tables[1]
    for row in volunteer_table.find_all('tr')[1:]:
        columns = row.find_all('td')
        if len(columns) > 1:
            name_tag = columns[0].find('a')
            name = name_tag.get_text(strip=True) if name_tag else '—'
            profile_link = name_tag['href'] if name_tag else '—'
            participant_id = profile_link.split('/')[-1] if profile_link != '—' else '—'
            # Извлекаем количество финишей и волонтёрств
            stats_div = columns[0].find('div', class_='user-stat')
            finishes = '—'
            volunteers = '—'
            if stats_div:
                stats_spans = stats_div.find_all('span')
                finishes = stats_spans[0].get_text(strip=True).split(' ')[0] if len(stats_spans) > 0 else '—'
                volunteers = stats_spans[1].get_text(strip=True).split(' ')[0] if len(stats_spans) > 1 else '—'
            # Клубы 
            club_tags = columns[0].find_all('span', class_='club-icon')
            clubs = ', '.join([club['title'] for club in club_tags]) if club_tags else '—'
            # Вторая колонка: роль волонтёра и информация о первом волонтёрстве
            volunteer_role_info = columns[1].find('div', class_='volunteer__role')
            if volunteer_role_info:
                # Извлекаем атрибут title для информации о первом волонтёрстве
                first_volunteer_tag = volunteer_role_info.find('span', class_='results_icon')
                first_volunteer_info = first_volunteer_tag['title'] if first_volunteer_tag else '—'
                # Извлекаем текст для роли волонтёра
                role_tag = volunteer_role_info.find_all('span')
                volunteer_role = role_tag[-1].get_text(strip=True) if role_tag else '—'
            else:
                first_volunteer_info = '—'
                volunteer_role = '—'
            volunteers_data.append([name, profile_link, participant_id, finishes, volunteers, clubs, volunteer_role, first_volunteer_info])
    return participants_data, volunteers_data

In [ ]:
run_protocol_link = 'https://5verst.ru/petergofaleksandriysky/results/12.10.2024/'
runners = parse_participant_and_volunteer_tables(run_protocol_link)[0]


In [ ]:
runners[10]

In [4]:
names = [card[1] for card in runners]
times = [card[9] for card in runners]
links = [card[2] for card in runners]

In [ ]:
names

## 17.10.2024 nrms-results parser for uknown runners

In [1]:
import pandas as pd
import os
import glob
import cv2
from datetime import datetime, timedelta
from jinja2 import Environment, FileSystemLoader
import random
import csv

In [2]:
run_protocol = "nrms_out.txt"

In [3]:
df = pd.read_csv(run_protocol, sep='\t', names=['position', 'time', 'id', 'name'])
times = df['time'].values.tolist()
ids = df['id'].values.tolist()
names = df['name'].values.tolist()

links = [fr'https://5verst.ru/userstats/{id[1:]}/' if id != 'A790000404' else '' for id in ids ]
# links

In [4]:
df

,position,time,id,name
0,1,00:19:19,A790126016,Артём ПКИН
1,2,00:20:24,A790092254,Дмитрий МЕНЬКОВ
2,3,00:20:28,A790097778,Василий ВАСИЛЬЧЕНКО
3,4,00:21:17,A790087192,Александр ТАРАСОВ
4,5,00:21:26,A790078287,Дмитрий ЧЕРНЫХ
...,...,...,...,...
103,104,00:47:34,A790167356,Елена ПЕРЕГУД
104,105,00:56:36,A790121455,Егор НИКИШОКИН
105,106,00:58:21,A790208777,Елена КОРИЧЕВА
106,107,01:00:12,A790089715,Александр ПАШИН


In [5]:
def str_to_time(strtime):
    return datetime.strptime(strtime,'%H:%M:%S')

In [11]:
first = str_to_time(df['time'][0])
first.minute, first.second

(19, 19)

In [19]:
shift = 22*60 + 45.7 - first.minute*60 - first.second # sec
vidcap = cv2.VideoCapture(r'D:\video\5verst\26.10.2024\VID20241026090554.mp4')

for i, timestring in enumerate(times[:3]):
    pt = datetime.strptime(timestring,'%H:%M:%S')
    total_seconds = pt.second + pt.minute*60 + pt.hour*3600
    
    vidcap.set(cv2.CAP_PROP_POS_MSEC, (total_seconds + shift - random.random()/3) * 1000)      # just cue to 20 sec. position
    success,image = vidcap.read()
    if success:
        cv2.imwrite(f"photos_128/{i+1}.jpg", image, [cv2.IMWRITE_JPEG_QUALITY, 50])

In [8]:
# Create the jinja2 environment.
current_directory = os.getcwd()
env = Environment(loader=FileSystemLoader(current_directory))

# Find all files with the j2 extension in the current directory
templates = glob.glob('*.j2') 

def render_template(filename):
    return env.get_template(filename).render(
    names=names,
    times=times,
    links=links,
    )

rendered_string = render_template(templates[0])


In [9]:
with open('out_127.html', 'w', encoding='utf-8') as file:
    file.write(rendered_string)

In [5]:
shift = 18*60 + 8 # sec
df['timecode'] = df['time'].apply(str_to_time) - timedelta(seconds=(shift+1))
df['timecodes'] = df['timecode'].apply(lambda x: x.strftime('%H:%M:%S'))
df['pos'] = df['position'].apply(str) + ')'
df

,position,time,id,name,timecode,timecodes,pos
0,1,00:19:43,A790163245,Александр АЛЕКСЕЕВ,1900-01-01 00:01:34,00:01:34,1)
1,2,00:20:08,A790193106,Горислав СКРИПКО,1900-01-01 00:01:59,00:01:59,2)
2,3,00:21:36,A790115199,Артём БУРНАЕВ,1900-01-01 00:03:27,00:03:27,3)
3,4,00:21:53,A790190949,Михаил ФЕДОРКОВ,1900-01-01 00:03:44,00:03:44,4)
4,5,00:22:10,A790200424,Вячеслав ЗАВАЛЬНЮК,1900-01-01 00:04:01,00:04:01,5)
...,...,...,...,...,...,...,...
103,104,00:40:05,A790199717,Ольга ФЕДОРКОВА,1900-01-01 00:21:56,00:21:56,104)
104,105,00:41:23,A790166347,Яков БУЧКИН,1900-01-01 00:23:14,00:23:14,105)
105,106,00:58:16,A790137828,Лев КОЧКИН,1900-01-01 00:40:07,00:40:07,106)
106,107,00:58:25,A790097975,Ксения КОЧКИНА,1900-01-01 00:40:16,00:40:16,107)


In [6]:
df[['timecodes', 'pos', 'name']].to_csv('timecodes.txt', index=False, sep='\t')

In [7]:
df.name.values

array(['Александр АЛЕКСЕЕВ', 'Горислав СКРИПКО', 'Артём БУРНАЕВ',
       'Михаил ФЕДОРКОВ', 'Вячеслав ЗАВАЛЬНЮК', 'Александр ТАРАСОВ',
       'Алексей ТЕРЕНТЬЕВ', 'Дмитрий МЕНЬКОВ', 'Василий АФАНАСЬЕВ',
       'Иван БОРОДИН', 'Силинский ЕВГЕНИЙ', 'Анна НАПАЛКОВА',
       'Юрий КУЗИН', 'Виталий ГОЛИНКО', 'Сергей КОЛЕСНИКОВ',
       'Константин МИРОЛЮБОВ', 'Дмитрий КАСИЛОВ', 'Александр БАРАНОВ',
       'Владимир ЧАЙЧЕНКО', 'Даниил ТУЗОВ', 'Александр КОЛЯНДА',
       'Валерий ТЮТЮННИК', 'Анатолий ПРОТОПОПОВ', 'Юлия ОССВЕТ',
       'Екатерина ШАХНЮК', 'Герман КАПИСТКА', 'Данила КОРНИКОВ',
       'Владимир СМИРНОВ', 'Фёдор КОЗЛОВ', 'Ольга КОЗЛОВА',
       'Сергей БИРЮКОВ', 'Александр ГРИГОРЬЕВ', 'Валерий РЕЗНИК',
       'Сергей БАЗЫЛЕВ', 'Игорь ИВАНОВ', 'Александр ГЕЙНТ',
       'Наталья БАРИНОВА', 'Евгений МАЛЬЦЕВ', 'Евгения РУМЯНЦЕВА',
       'Екатерина МУШИНСКАЯ', 'Екатерина БАТОВА', 'Надежда ГУКОВА',
       'Анастасия ПЕТРОВА', 'Алиса ПЕТРОВА', 'Александр СИМКИН',
       'Елена КОСТАМАН

## 24.10.2024 youtube-dl - save video frames from online video

In [3]:
import cv2
import numpy as np
import yt_dlp as youtube_dl

In [11]:
video_url = 'https://youtu.be/qyy7YpwroPw'

ydl_opts = {}

# create youtube-dl object
ydl = youtube_dl.YoutubeDL(ydl_opts)

# set video url, extract video information
info_dict = ydl.extract_info(video_url, download=False)

# get video formats available
formats = info_dict.get('formats',None)

formats

# for f in formats:

#     # I want the lowest resolution, so I set resolution as 144p
#     if f.get('format_note',None) == '144p':

#         #get the video url
#         url = f.get('url',None)

#         # open url with opencv
#         cap = cv2.VideoCapture(url)

#         # check if url was opened
#         if not cap.isOpened():
#             print('video not opened')
#             exit(-1)

#         while True:
#             # read frame
#             ret, frame = cap.read()

#             # check if frame is empty
#             if not ret:
#                 break

#             # display frame
#             cv2.imshow('frame', frame)

#             if cv2.waitKey(30)&0xFF == ord('q'):
#                 break

#         # release VideoCaptureq
#         cap.release()

# cv2.destroyAllWindows()

[youtube] Extracting URL: https://youtu.be/qyy7YpwroPw
[youtube] qyy7YpwroPw: Downloading webpage
[youtube] qyy7YpwroPw: Downloading ios player API JSON
[youtube] qyy7YpwroPw: Downloading mweb player API JSON
[youtube] qyy7YpwroPw: Downloading m3u8 information


[{'format_id': 'sb1',
  'format_note': 'storyboard',
  'ext': 'mhtml',
  'protocol': 'mhtml',
  'acodec': 'none',
  'vcodec': 'none',
  'url': 'https://i.ytimg.com/sb/qyy7YpwroPw/storyboard3_L1/M$M.jpg?sqp=-oaymwENSDfyq4qpAwVwAcABBqLzl_8DBgi4t_-FBg==&sigh=rs$AOn4CLBj2HpUTXpviSDNzBOH2HFjbfgbgw',
  'width': 25,
  'height': 45,
  'fps': 0.2052980132450331,
  'rows': 10,
  'columns': 10,
  'fragments': [{'url': 'https://i.ytimg.com/sb/qyy7YpwroPw/storyboard3_L1/M0.jpg?sqp=-oaymwENSDfyq4qpAwVwAcABBqLzl_8DBgi4t_-FBg==&sigh=rs$AOn4CLBj2HpUTXpviSDNzBOH2HFjbfgbgw',
    'duration': 302.0}],
  'resolution': '25x45',
  'aspect_ratio': 0.56,
  'filesize_approx': None,
  'http_headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'Accept-Language': 'en-us,en;q=0.5', 'Sec-Fetch-Mode': 'navigate'},
  'audio_ext': 'none',
  'video_ext': 'n

In [12]:
for _ in formats:
    print(_)

{'format_id': 'sb1', 'format_note': 'storyboard', 'ext': 'mhtml', 'protocol': 'mhtml', 'acodec': 'none', 'vcodec': 'none', 'url': 'https://i.ytimg.com/sb/qyy7YpwroPw/storyboard3_L1/M$M.jpg?sqp=-oaymwENSDfyq4qpAwVwAcABBqLzl_8DBgi4t_-FBg==&sigh=rs$AOn4CLBj2HpUTXpviSDNzBOH2HFjbfgbgw', 'width': 25, 'height': 45, 'fps': 0.2052980132450331, 'rows': 10, 'columns': 10, 'fragments': [{'url': 'https://i.ytimg.com/sb/qyy7YpwroPw/storyboard3_L1/M0.jpg?sqp=-oaymwENSDfyq4qpAwVwAcABBqLzl_8DBgi4t_-FBg==&sigh=rs$AOn4CLBj2HpUTXpviSDNzBOH2HFjbfgbgw', 'duration': 302.0}], 'resolution': '25x45', 'aspect_ratio': 0.56, 'filesize_approx': None, 'http_headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'Accept-Language': 'en-us,en;q=0.5', 'Sec-Fetch-Mode': 'navigate'}, 'audio_ext': 'none', 'video_ext': 'none', 'vbr': 0, 'abr': 0, 'tbr': None, 'f